In [2]:
import os
from pathlib import Path
import sys

if "__project_dir__" not in globals():
    __project_dir__ = Path.cwd().parents[1].resolve()

sys.path.append(__project_dir__)
os.chdir(__project_dir__)

In [3]:
import numpy as np
import os
import pandas as pd
from pathlib import Path
from scipy.spatial.distance import pdist, squareform
from src.model_loader import ModelLoader
import sys
import tidytcells as tt

if "__project_dir__" not in globals():
    __project_dir__ = Path.cwd().parents[1].resolve()

sys.path.append(__project_dir__)
os.chdir(__project_dir__)

In [17]:
functional_trbvs = pd.DataFrame(
    tt.tcr.query(precision="gene", functionality="F", contains="TRBV"), columns=["TRBV"]
).sort_values(by="TRBV", ignore_index=True)

In [18]:
savedir = Path("v_gene_representations")

if not savedir.is_dir():
    savedir.mkdir()

In [19]:
model = ModelLoader(
    Path("/home/yutanagano/UCLOneDrive/MBPhD/projects/tcr_embedder/model_saves/BCDRBERT_ACL_small_double_censoring")
)

embeddings = model.embed(functional_trbvs)
cdist = squareform(pdist(embeddings))

In [22]:
cdist = pd.DataFrame(cdist, columns=functional_trbvs["TRBV"].values, index=functional_trbvs["TRBV"].values)

In [25]:
cdist.to_csv(savedir/"ml_trbv_dists.csv")